In [1]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, pipeline
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.postprocessor import SimilarityPostprocessor
from huggingface_hub import snapshot_download
from vllm import LLM, SamplingParams
from sentence_transformers import CrossEncoder
from langchain_community.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from docx import Document as DocxDocument
import pandas as pd

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/user1/environments/hack/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [28]:
knowledge_base = None

In [29]:
# Функция для разбиения файлов на чанки
def file_to_chunks_with_splitter(file_name, sep, chunk_size, chunk_overlap):
    file_ext = file_name.split('.')[-1]
    file_path = file_name
    overall_chunks = []

    if file_ext == 'txt':
        loader = TextLoader(file_path, encoding='utf-8')
        text_content = loader.load()
    elif file_ext == 'docx':
        loader = DocxDocument(file_path)
        text_content = []
        for para in loader.paragraphs:
            if para.text.strip():
                text_content.append(para.text.strip())
        for table in loader.tables:
            for row in table.rows:
                row_data = [cell.text.strip() for cell in row.cells]
                text_content.append(sep.join(row_data))
    elif file_ext == 'pdf':
        loader = PyPDFLoader(file_path)
        text_content = [page.extract_text() for page in loader.load() if page.extract_text() is not None]
    else:
        raise ValueError("Unsupported file format")

    text_splitter = RecursiveCharacterTextSplitter(
        separators=sep,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
        add_start_index=False
    )

    for content in text_content:
        if content.strip():
            chunks = text_splitter.split_text(content)
            overall_chunks.extend([chunk.strip() for chunk in chunks if chunk.strip()])

    return overall_chunks

In [30]:
# Функция для загрузки и обработки проекта с вложенными файлами 2048 128
def load_project_files(project_dir, 
                       sep='\n', 
                       chunk_size=4096, 
                       chunk_overlap=128):
    documents = []
    for root, _, files in os.walk(project_dir):
        for file in files:
            if file.startswith('.'):
                continue
            file_path = os.path.join(root, file)
            try:
                chunks = file_to_chunks_with_splitter(file_path, sep, chunk_size, chunk_overlap)
                for chunk in chunks:
                    metadata = {"название документа": file_path}
                    documents.append(Document(
                        text=chunk,
                        metadata=metadata,
                        excluded_embed_metadata_keys=["название документа"]
                    ))
            except Exception as e:
                print(f"Ошибка при обработке файла {file_path}: {e}")
    return documents

In [31]:
# Создание базы знаний из загруженных документов
def create_knowledge_base(base_dir='/home/user1/GreenQuery/datasets/knowledge_data_del/'):
    docs = load_project_files(base_dir)
    if docs:
        index = VectorStoreIndex.from_documents(documents=docs, show_progress=True)
        retriever = index.as_retriever(similarity_top_k=7, 
                                       node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.85)])
        return retriever
    else:
        print("Не удалось загрузить документы для базы знаний.")
        return None

In [32]:
# Функция для загрузки или повторного использования базы знаний
def load_or_create_knowledge_base(prj_dir):
    global knowledge_base
    if knowledge_base is None:
        knowledge_base = create_knowledge_base(prj_dir)
    return knowledge_base

In [33]:
# Инициализация моделей
snapshot_download(
    repo_id="IlyaGusev/saiga_llama3_8b",
    revision="main_vllm",
    local_dir="/home/user1/GreenQuery/llm_saiga_src"
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/home/user1/GreenQuery/llm_saiga_src'

In [34]:
tokenizer = AutoTokenizer.from_pretrained("/home/user1/GreenQuery/llm_saiga_src")
llm = LLM(
    model="/home/user1/GreenQuery/llm_saiga_src",
    dtype=torch.float16,
    gpu_memory_utilization=0.65,
    max_seq_len_to_capture=8192
)

WARNING 10-27 07:41:11 config.py:1668] Casting torch.bfloat16 to torch.float16.
INFO 10-27 07:41:11 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/home/user1/GreenQuery/llm_saiga_src', speculative_config=None, tokenizer='/home/user1/GreenQuery/llm_saiga_src', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/user1/Gre

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 221.44 MiB is free. Including non-PyTorch memory, this process has 31.51 GiB memory in use. Of the allocated memory 30.88 GiB is allocated by PyTorch, with 31.38 MiB allocated in private pools (e.g., CUDA Graphs), and 80.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3')

In [ ]:
summarizer = pipeline("summarization", model="IlyaGusev/rut5_base_sum_gazeta")

In [ ]:
# Классификация запроса: суммаризация или вопрос
def classify_query(query):
    if "суммаризация" in query.lower() or "обзор" in query.lower():
        return "summarization"
    return "question"

In [ ]:
# Функция для реранкинга топовых документов
def top_k_rerank(query: str, retriever, reranker, top_k: int = 2):
    documents = retriever.retrieve(query)
    relevant_score = documents[0].score
    print(f'Наивысшее значение релевантности документов: {relevant_score}')

    candidate_texts = [x.text for x in documents]
    candidate_names = [x.metadata['название документа'] for x in documents]

    rerank_scores = reranker.predict(list(zip([query] * len(candidate_texts), candidate_texts)))
    ranked_indices = np.argsort(rerank_scores)[::-1]

    names = [candidate_names[i] for i in ranked_indices][:top_k]
    texts = [candidate_texts[i] for i in ranked_indices][:top_k]

    return names, texts, relevant_score

In [ ]:
# Функция для генерации примеров
def get_example_prompts(num_examples=3):
    answer_csv_path = '/home/user1/GreenQuery/datasets/metrics/train.csv'
    answer_data = pd.read_csv(answer_csv_path, sep='\t')
    examples = answer_data.sample(n=num_examples)  # Берем случайные примеры
    example_text = ""
    for _, row in examples.iterrows():
        question = row['Вопрос']
        answer = row['Ответ']
        example_text += f"Вопрос: {question}\nОтвет: {answer}\n\n"
    return example_text


In [ ]:
def vllm_infer(tokenizer, wrapped_llm, texts, query,
               temperature: float = 0.1,
               top_p: float = 0.95,
               top_k: int = 50,
               max_tokens: int = 2048, # 1024
               repetition_penalty: float = 1.05,
               presence_penalty: float = 0.5,
               frequency_penalty: float = 0.2,
               stop=["Я не могу ответить на ваш вопрос.", "Ответ окончен"]):

    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    
    example_prompts = get_example_prompts(num_examples=3)
    
    user_prompt = '''Ниже приведены примеры вопросов и ответов. Постарайся придерживаться их структуры и уровня детализации.
        {examples}
        
        Используй только следующий контекст, чтобы подробно ответить на вопрос в конце.
        Пожалуйста, укажи все важные детали, присутствующие в контексте.
        Если контекст не соотносится с вопросом, скажи, что ты не можешь ответить на данный вопрос.
        Если вопрос не относится к экологической тематике, выведи фразу "Я не могу ответить на ваш вопрос." и не выводи ничего больше.
        
        Контекст:
        =========== 
        {texts} 
        =========== 
        
        Вопрос:
        =========== 
        {query}'''.format(examples=example_prompts, texts=texts, query=query)

    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        max_tokens=max_tokens,
        repetition_penalty=repetition_penalty,
        presence_penalty=presence_penalty,
        frequency_penalty=frequency_penalty,
        stop=stop
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    prompt = wrapped_llm.llm_engine.tokenizer.tokenizer.apply_chat_template(
        conversation=messages, add_generation_prompt=True, tokenize=False
    )
    prompts = [prompt]
    outputs = wrapped_llm.generate(prompts, sampling_params)
    
    answers = [output.outputs[0].text for output in outputs]
    torch.cuda.empty_cache()
    return answers


In [35]:
# Функция для генерации ответа с проверкой на релевантность
def response(query, prj_dir, knowledge_retriever, reranker, tokenizer, llm):
    query_type = classify_query(query)

    # Если запрос на суммаризацию загруженного проекта
    if query_type == "summarization":
        docs = load_project_files(prj_dir)
        return summarize_documents(docs)
    
    # Если запрос относится к базе знаний
    elif "база знаний" in query.lower():
        names, chunks, relevant_score = top_k_rerank(query, knowledge_retriever, reranker)
        if relevant_score >= 0.5:
            answer = vllm_infer(tokenizer, llm, ' '.join(chunks), query)
            return answer[0] if answer[0] != 'Я не могу ответить на ваш вопрос.' else "Нет подходящего ответа."
        else:
            return 'Запрос не найден в базе знаний.'

    # Если запрос касается загруженного проекта
    else:
        docs = load_project_files(prj_dir)
        prj_index = VectorStoreIndex.from_documents(documents=docs, show_progress=True)
        prj_retriever = prj_index.as_retriever(similarity_top_k=7,
                                               node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.85)])
        names, chunks, relevant_score = top_k_rerank(query, prj_retriever, reranker)
        if relevant_score >= 0.5:
            answer = vllm_infer(tokenizer, llm, ' '.join(chunks), query)
            return answer[0] if answer[0] != 'Я не могу ответить на ваш вопрос.' else "Нет подходящего ответа."
        else:
            return 'Запрос не найден в загруженном проекте.'

In [36]:
arr = [
    "Объяснить, что такое источник выбросов, источник выделения.",
    "Указать этапы разработки проекта начиная с получения в работу.",
    "Расписать состав тома ПДВ",
    "Как присваиваются номера источников выбросов предприятиям при разработке проекта впервые, повторно.",
    "Что такое газоочистные установки? Приведите их примеры. Какие документы разрабатываются для них. Что добавляется в проект при появлении в нем ГОУ?",
    "Что должно быть отображено на ситуационных картах?",
    "Что такое ОБУВ, ПДК? Какие ПДК вы знаете?",
    "Что такое превышение предельно допустимой концентрации в проекте ПДВ?",
    "Напишите приблизительный перечень источников выбросов загрязняющих веществ от сельскохозяйственного предприятия (занимается хранением и транспортировкой зерна)",
    "Какие этапы согласования проходит проект ПДВ?",
    "Какие параметры ИЗАВ и ГВС необходимы для проведения расчета в УПРЗА?",
    "Кому необходимо проведение инвентаризации стационарных источников и выбросов загрязняющих веществ в атмосферный воздух?",
    "Каков порядок нумерации источников выбросов загрязняющих веществ (ИЗАВ) при проведении корректировки инвентаризации на объекте?",
    "Что такое передвижной ИЗАВ и каков порядок его учета при проведении инвентаризации?",
    "Что такое нестационарность выбросов и как она отражается в отчете об инвентаризации?",
    "Какие есть особенности при проведении инвентаризации для строящихся и реконструируемых объектов?",
    "Какие есть способы определения количественных характеристик выбросов загрязняющих веществ?",
    "Использование каких методик допускается при проведении расчетов выбросов (общая характерная черта всех методик)?",
    "Каков принцип определения геометрических характеристик источников выбросов?",
    "В каких случаях объектам требуется корректировка данных инвентаризации выбросов?",
    "В чем заключается разница между разработкой и установлением нормативов допустимых выбросов загрязняющих веществ в атмосферный воздух для объектов I и III категории и объектов II категории.",
    "Какой обязательный состав и что в себя включает проект нормативов допустимых выбросов?",
    "Что такое гигиенические нормативы, каких видов они бывают и как применимы при разработке и установлении нормативов допустимых выбросов загрязняющих веществ в атмосферный воздух?",
    "Что такое зона влияния загрязняющего вещества?",
    "Что такое перспектива развития в рамках разработки и утверждения нормативов допустимых выбросов загрязняющих веществ в атмосферный воздух?",
    "Какая дополнительная информация приводится в составе проекта НДВ (ПДВ) на случай возникновения неблагоприятных метеорологических условий?"
]

In [37]:
prj_dir = '/home/user1/GreenQuery/datasets/knowledge_data_del/'

# Получение базы знаний, если она ещё не была создана
knowledge_base_instance = load_or_create_knowledge_base(prj_dir)

Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Учебник-Экология.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/akselevich_torgunakova.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Закон от 04_05_1999 N 96-ФЗ Об охране атмосферного воздуха (с изменениями на 8 августа 2024 года)_Текст.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Письмо_ Минприроды России от 16.11.2021 N 20-47_35535.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Методика расчета выделений загрязняющих веществ в атмосферу от стационарных дизел

incorrect startxref pointer(1)


Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Методика проведения инвентаризации выбросов загрязняющих веществ в атмосферу для авторемонтных предприятий (расчетным методом). М, 1998.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Методические указания по определению выбросов загрязняющих веществ в атмосферу из резервуаров Новополоцк 1997.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/ГОСТ Р 58577-2019. Национальный стандарт Российской Федераци.pdf: 'Document' object has no attribute 'extract_text'
Ошибка при обработке файла /home/user1/GreenQuery/datasets/knowledge_data_del/Нормативка и учебники/Методика расчета выделений (выбросов)загрязняющих веществ в атмосферу при нанесении лакокрасочных материалов.pdf: 'Document

Parsing nodes:   0%|          | 0/47748 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/833 [00:00<?, ?it/s]

In [40]:
for i in range(len(arr)):
    query_knowledge_base = arr[i]
    print(response(query_knowledge_base, knowledge_base_instance, reranker, tokenizer, llm))

NameError: name 'vllm' is not defined

In [ ]:
prj_dir = '/home/user1/GreenQuery/datasets/Данные для тестирования/Проект 2 (для QnA)/Проект Word'

In [ ]:
arr_1 = [
    "Какой деятельностью занимается ООО «Эко Агро»?",
    "На скольких производственных площадках осуществляет деятельность ООО «Эко Агро»?",
    "В каком направлении расположена территория для выпаса сельскохозяйственных животных?",
    "На каком расстоянии расположена нормируемая территория в северо-восточном направлении?",
    "Какое разрешенное использование у кадастрового номера?",
    "Где находится ближайшая жилая зона?",
    "Какой адрес у кадастрового номера?",
    "На каком расстоянии расположена ближайшая жилая зона в северо-восточном направлении?",
    "Сколько метров составляет ориентировочная санитарно-защитная зона для объектов и производств агропромышленного комплекса?",
    "В каком направлении расположена территория для выпаса сельскохозяйственных животных?",
    "На чем работает котел Дон-60?",
    "В чем привозится газ?",
    "Что располагается на площадке ток асфальтированный?",
    "От чего осуществляется выброс загрязняющих веществ?",
    "Какой код у вещества Пыль зерновая?",
    "Как учитывается проезд автотранспорта по территории предприятия?",
    "Через что происходит выброс загрязняющих веществ в атмосферный воздух?",
    "Согласно какому методическому пособию выбросы отсутствуют при обработки стали без СОЖ?",
    "Какие загрязняющие вещества выделяются от работы сварочного поста?",
    "Для чего предназначено административное здание?",
    "От какого процесса происходит выброс загрязняющих веществ?",
    "Сколько источников загрязнения атмосферного воздуха выявлено в результате хозяйственной деятельности ООО «Эко Агро»",
    "Куда наносятся источники загрязнения предприятия?",
    "В каких случаях требуется корректировка данных инвентаризации выбросов объекта ОНВ?",

]

In [ ]:
for i in range(len(arr_1)):
    query_knowledge_base = arr_1[i]
    print(response(query_knowledge_base, knowledge_base_instance, reranker, tokenizer, llm))